In [122]:
import cirq
import numpy as np
import random

#Setup
n_qbits = 2

qbit = cirq.LineQubit.range(n_qbits)
ancilla = cirq.NamedQubit("A")

def make_oracle(qubits, ancilla, xprime):
    """Implements the function {f(x) = 1 if x == x', f(x) = 0 if x != x'}."""
    # For x' = (1, 1), the oracle is just a Toffoli gate.
    # For a general x', we negate the zero bits and implement a Toffoli.

    #Flip all qubits not marked by xprime
    for i in range(len(qubits)):
        if not xprime[i]:
            yield cirq.X(qubits[i])
    
    yield cirq.TOFFOLI(qubits[0],qubits[1],ancilla)# oracle for 2 qubit search - mark chosen bit
    
    #Flip all qubits not marked by xprime
    for i in range(len(qubits)):
        if not xprime[i]:
            yield cirq.X(qubits[i])

In [108]:
def grover_iteration(qubits, ancilla, oracle):
    #perform 1 round
    circuit = cirq.Circuit()
    
    #Superposition all
    for q in qubits:
        circuit.append(cirq.H(q))
        
    #Flip anc to |-) state
    circuit.append(cirq.X(ancilla))
    circuit.append(cirq.H(ancilla))
    
    #Oracle Time
    circuit.append(oracle)
    
    
    #Grover Operator - flips state over s and then flips it over its initial state
    for q in qubits:
        circuit.append(cirq.H(q))
        circuit.append(cirq.X(q))
    circuit.append(cirq.H(qubits[1]))
    circuit.append(cirq.CNOT(qubits[0],qubits[1]))
    circuit.append(cirq.H(qubits[1]))
    for q in qubits:
        circuit.append(cirq.X(q))
        circuit.append(cirq.H(q))
        
    # Measure
    circuit.append(cirq.measure(*qubits, key="result"))

    return circuit

In [119]:
#randomize bitstring
xprime = [0]*n_qbits
for i in range(n_qbits):
    xprime[i] = random.randint(0,1)

print(xprime)

[0, 1]


In [120]:
#Build Circuit
oracle = make_oracle(qbit, ancilla, xprime)
circuit = grover_iteration(qbit, ancilla, oracle)
print(circuit)


0: ───H───X───@───X───H───X───@───X───H───────M('result')───
              │               │               │
1: ───H───────@───H───X───H───X───H───X───H───M─────────────
              │
A: ───X───H───X─────────────────────────────────────────────


In [121]:
"""Simulate the circuit for Grover's algorithm and check the output."""
# Helper function.
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)

# Sample from the circuit a couple times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=10)

# Look at the sampled bitstrings.
frequencies = result.histogram(key="result", fold_func=bitstring)
print('Sampled results:\n{}'.format(frequencies))

# Check if we actually found the secret value.
most_common_bitstring = frequencies.most_common(1)[0][0]
print("\nMost common bitstring: {}".format(most_common_bitstring))
print("Found a match? {}".format(most_common_bitstring == bitstring(xprime)))

Sampled results:
Counter({'01': 10})

Most common bitstring: 01
Found a match? True
